In [1]:
import torch
import numpy as np
import pandas as pd
import codecs
import re
import nltk
import random

from nltk.stem import WordNetLemmatizer

from random import shuffle

from collections import Counter

from numpy import array

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.preprocessing import OneHotEncoder

from torch import tensor
from torch import nn
from torch import optim
from torch.autograd import Variable
import torch.utils.data.dataloader as dataloader
from torch.utils.data import Dataset

from scipy.stats import entropy


# Parameters

In [2]:
# Number of recognized words you put in input
nb_input = 100 #8414 max

# Number of classe, constant
nb_output = 6

# Number of hidden layers
nb_hidd_lay = 5
hidden_size = 10

# Learning rate
lr = 0.0001

# Number of epochs
nb_epochs = 10

# Random seed, don't change it if you don't know what it is
random_seed = 42

nb_batchs = 4

use_cuda = torch.cuda.is_available()

In [3]:
def seeding_random():
    random.seed(random_seed)
    torch.manual_seed(random_seed)
    np.random.seed(random_seed)
    if use_cuda:
        torch.cuda.manual_seed_all(random_seed)
        torch.cuda.manual_seed(random_seed)

torch.backends.cudnn.deterministic=True

In [4]:
# Lemmatisation
# Working on list_of_words
# TODO: it doesn't work
def lemm(phrase):
    lemmer = WordNetLemmatizer()
    lemmed = []
    for word in phrase:
        lemmed.append(lemmer.lemmatize(word, pos="v"))
    return lemmed

## Dataloader implementation

In [5]:
class QuestionDataset(Dataset):
    def __init__(self, file, nb_most_commons):
        # Encoding in windows-1252, utf-8 generate error on some char
        file = codecs.open(file, "r+","windows-1252")
        train_data = []
        for line in file.readlines():
            train_data.append(line)

        # Divided file into 2 list:
        # questions = list of questions 
        # labels = list of labels

        questions = []
        labels = []

        # Black list
        regex = re.compile('[@_!#$%^&*()<>?/\|}{~:]')

        (train_data[0]).split()[0]
        for string in train_data:
            question_str = []
            for x in lemm(string.split()[1:]):
                question_str.append(x.lower())
            labels.append(string.split()[0])
            questions.append(question_str)

        self.add_tag(questions)
        
        # Vocabulary of unique words
        data = []
        data.append('<unk>')
        for q in questions:
            for word in q:
                data.append(word)
        #print(len(data))
        scv = np.array(data)
        unik, counts = np.unique(scv,return_counts=True)
        vocabulary = {}
        for i in range(0, len(unik)):
            vocabulary[unik[i]] = counts[i]
    
        word_list = list([x[0] for x in Counter(vocabulary).most_common(nb_most_commons+2)])
        # We add the unk word for future purpose.
        word_list.append('<unk>')
        words_array = np.array(word_list)
        
        # Integer encoding with OneHotEncoder
        words_tre = words_array.reshape(len(words_array),1)
        one_hot_encoder = OneHotEncoder(sparse=False)
        onehot_encoded = one_hot_encoder.fit_transform(words_tre)
        # Creating a dictionnary of word and its one hot array
        self.words_onehoted = {}
        for i in range(0, len(words_array)):
            self.words_onehoted[word_list[i]] = onehot_encoded[i]

        # One hot categories
        self.categories_onehoted = {}
        self.categories_onehoted['ABBR'] = 0 # Abbreviation
        self.categories_onehoted['ENTY'] = 1 # Entity
        self.categories_onehoted['DESC'] = 2 # Description
        self.categories_onehoted['HUM']  = 3 # Human
        self.categories_onehoted['LOC']  = 4 # Location
        self.categories_onehoted['NUM']  = 5 # Numeric
        
        self.batch_data = []
        for num_question in range(len(questions)):
            # Construction of question_onehot list.
            question_onehot = [self.get_onehot_word(word) for word in questions[num_question]]

            # Construction of category_onehot.
            category = labels[num_question].partition(':')[0]
            category_onehot = self.get_onehot_category(category)
            self.batch_data.append([(question_onehot), (category_onehot)])
    
        
    
    # Function to get the corresponding one hot list for a category.
    def get_onehot_category(self, category):
        return self.categories_onehoted[category]


    # Function to get the corresponding one hot list for a word.
    def get_onehot_word(self, word):
        if word in self.words_onehoted:
            return list(self.words_onehoted[word])
        else:
            return list(self.words_onehoted['<unk>'])

    

    # Method to add tags begin and end to phrases list.
    # /!\ 
    # WARNING : this method need to be executed only ONE time.
    # /!\
    def add_tag(self, question_list):
        for i in range(0, len(question_list)):
            if question_list[i][0] != '<bos>' :
                question_list[i].insert(0, '<bos>')
                question_list[i].append('<eos>')
                
    def __len__(self):
        return len(self.batch_data)

    def __getitem__(self, idx):
        seeding_random()
        return self.batch_data[idx]
    
def pad_collate(batch):
    max_length = max([len(q[0]) for q in batch])

    inputs = torch.FloatTensor([[[0. for _ in range(len(x[0][0]))] for i in range(max_length-len(x[0]))]+x[0] for x in batch])
    outputs = torch.LongTensor([x[1] for x in batch])
    
    return inputs, outputs
    

In [6]:

seeding_random()
training_set = QuestionDataset("train_all.label", nb_input-3)
seeding_random()

test_set = QuestionDataset("TREC_test.label", nb_input-3)
seeding_random()

# Création du DataLoader
dataloader_args = dict(shuffle=True, batch_size=nb_batchs, num_workers=1,
                       pin_memory=True, worker_init_fn=seeding_random(), collate_fn=pad_collate)# if use_cuda else dict(collate_fn=pad_collate)
seeding_random()

train_loader = dataloader.DataLoader(training_set, **dataloader_args)
seeding_random()

dataloader_args_test = dict(shuffle=False, batch_size=nb_batchs, num_workers=1,
                       pin_memory=True, worker_init_fn=seeding_random(), collate_fn=pad_collate)# if use_cuda else dict(collate_fn=pad_collate)
seeding_random()

test_loader = dataloader.DataLoader(test_set, **dataloader_args_test)
seeding_random()



# RNN implementation
Using ReLU, and CrossEntropy

In [20]:
class RNN(nn.Module):
    def __init__(self, nb_inputs, nb_layers, nb_neurons, nb_outputs, learning_rate):
        super(RNN, self).__init__()
        
        # Applying RNN layer, and softmax then
        self.rnn = nn.RNN(input_size=nb_inputs, num_layers=nb_layers,
                   hidden_size=nb_neurons, dropout=0., batch_first=True, nonlinearity='relu')
        self.inter = nn.Linear(nb_neurons, nb_outputs)
        self.sm = nn.Softmax(dim=1)
        
        # Other usefull variables here
        self.optimizer = optim.Adam(self.parameters(), lr=learning_rate)
        self.input_dim = nb_inputs
        self.output_dim = nb_output
        self.nb_layers = nb_layers
        self.nb_neurons = nb_neurons
        
    def forward(self, inputs):
        h0 = torch.zeros(self.nb_layers, inputs.size(0), self.nb_neurons)
        if use_cuda:
            h0 = h0.to("cuda")
        x, hn = self.rnn(inputs, h0)
        
        x = self.inter(hn[0])
        #print(x)
        #x = tensor([list(i[-1]) for i in x])
        #print(x)
        x = self.sm(x)
        return x

# End of the class RNN

# Now let's define learn(), which learn a RNN some data
def learn(rnn, data_loader, num_epochs=1):
    device = torch.device("cuda" if use_cuda else "cpu")
    if use_cuda:
        torch.set_default_tensor_type('torch.cuda.FloatTensor')
        rnn.cuda(device)
    
    # Preparing
    rnn.train()
    losses = []
    criterion = nn.CrossEntropyLoss()
    

    for epoch in range(num_epochs):
        total_correct = 0
        total_target = 0
        
        for batch_idx, (data, target) in enumerate(data_loader):
            data, target = data.to(device), target.to(device)
            
            output = rnn(data)
            #print(output)
            #print(target)
            loss = criterion(output, target)
            rnn.optimizer.zero_grad()
            loss.backward()
            rnn.optimizer.step()
            losses.append(loss.data.item())
            
            # Get the Accuracy
            
            _, predicted = torch.max(output.data, dim=1)
            correct = (predicted == target).sum().item()
            total_correct += correct
            total_target += target.size(0)
            
            # Print the progress
            if batch_idx % 100 == 0 or batch_idx % 100 == 1 or batch_idx == len(data_loader)-1:
                  print('\r Train Epoch: {} [{}/{} ({:.0f}%)]\t Loss: {:.6f}\t Accuracy: {}'.format(
                    epoch, 
                    batch_idx * len(data), 
                    len(data_loader.dataset),
                    100. * batch_idx / len(data_loader), 
                    loss.data.item(),
                    (total_correct / total_target) * 100),
                    end='')
        print()
        
    # Return losses list, you can print them later if you want
    return losses

# return (rightAnswer, falseAnswer, entropy_list)
def getEfficience(rnn, batch_list) :
    entropy_list = []
    total_correct = 0
    total = 0
    device = torch.device("cuda" if use_cuda else "cpu")
    for (data, target) in batch_list :
        data, target = data.to(device), target.to(device)
        out = rnn(data).data
        
        _, predicted = torch.max(out.data, dim=1)
        correct = (predicted == target).sum().item()
        total_correct += correct
        total += target.size(0)

        value, counts = np.unique(out, return_counts=True)
        entropy_list.append(entropy(out, base=None))

    return (total_correct, total_target-total_correct, entropy_list)

# Using the RNN

In [8]:
seeding_random()

rnn = RNN(nb_inputs = nb_input, nb_layers=nb_hidd_lay,
          nb_neurons=hidden_size, nb_outputs=nb_output, learning_rate=lr)
if use_cuda:
    rnn = rnn.to("cuda")

seeding_random()


with torch.enable_grad():
    losses = learn(rnn, train_loader, nb_epochs)
    print("Done :)")

 Train Epoch: 0 [15448/15452 (100%)]	 Loss: 1.784005	 Accuracy: 18.91664509448615
 Train Epoch: 1 [15448/15452 (100%)]	 Loss: 1.672840	 Accuracy: 23.265596686513074
 Train Epoch: 2 [15448/15452 (100%)]	 Loss: 1.625977	 Accuracy: 23.084390370178618
 Train Epoch: 3 [15448/15452 (100%)]	 Loss: 1.576590	 Accuracy: 25.23297954957287
 Train Epoch: 4 [15448/15452 (100%)]	 Loss: 1.612147	 Accuracy: 32.96660626456122
 Train Epoch: 5 [15448/15452 (100%)]	 Loss: 1.706352	 Accuracy: 37.943308309603935
 Train Epoch: 6 [15448/15452 (100%)]	 Loss: 1.758542	 Accuracy: 42.577012684442145
 Train Epoch: 7 [15448/15452 (100%)]	 Loss: 1.667837	 Accuracy: 44.84209163862283
 Train Epoch: 8 [15448/15452 (100%)]	 Loss: 1.513853	 Accuracy: 45.94874449909397
 Train Epoch: 9 [15448/15452 (100%)]	 Loss: 1.698738	 Accuracy: 46.440590214858915
Done :)


## Error curve

In [11]:
from scipy.signal import savgol_filter
import ipywidgets as widgets
from ipywidgets import interact
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.pyplot import imshow


def update_losses(smooth=1):
    x = np.linspace(0, len(losses), len(losses))
    fig = plt.figure(figsize=(13, 8)) 
    ax = fig.add_subplot(1,1,1)
    cnn_line, = ax.plot(x, losses)
    cnn_line.set_ydata(savgol_filter(losses, smooth, 3))
    
interact(update_losses, smooth=(5, 201, 2));


interactive(children=(IntSlider(value=5, description='smooth', max=201, min=5, step=2), Output()), _dom_classe…

# Analysis on test set

In [22]:
rnn.eval()

seeding_random()

final_results = getEfficience(rnn, train_loader)
total = sum(final_results[:2])

correct_train = final_results[0]/total*100
false_train = final_results[1]/total*100


print("Congratulations!")
print("On the training set:")
print("Corrects: " + str(correct_train) + "%")
print("False:    " + str(false_train) + "%")
print()

seeding_random()

final_results = getEfficience(rnn, test_loader)
total = sum(final_results[:2])

correct = final_results[0]/total*100
false = final_results[1]/total*100
mean_entropies = sum(final_results[2])/len(final_results[2])
print(final_results[2])
#Entropy mean might be near to zero
print("On the test set:")
print("Moyenne des entropies: " + str(mean_entropies))
print("Corrects: " + str(correct) + "%")
print("False:    " + str(false) + "%")

mean_entropies = -1
print()
print("A présent, tu peux copier-coller ça dans le doc sur le drive :)")
print(str(nb_input)+"\t"+str(lr)+"\t"+str(nb_epochs)+"\t"+str(nb_hidd_lay)+"\t"+str(hidden_size)+"\t"+str(nb_batchs)+"\t\t"+str(mean_entropies)+"\t"+str(correct_train)+"%\t"+str(correct)+"%")
print()



Congratulations!
On the training set:
Corrects: 46.3888169816205%
False:    53.61118301837949%

[array([ 0.02737719,  1.08859992,  0.58416933,  1.07360196,  0.58113724,
        0.56308007], dtype=float32), array([ 0.05083909,  1.18554378,  0.6013732 ,  1.10491848,  0.5934025 ,
        0.56890309], dtype=float32), array([  2.29224914e-23,   7.55336821e-01,   5.29539597e-04,
         1.28820717e+00,   1.47581259e-05,   1.59848496e-05], dtype=float32), array([ 0.01424396,  0.76633573,  0.60259211,  1.19096816,  0.50275183,
        0.50325227], dtype=float32), array([  4.91164986e-30,   1.05162013e+00,   6.90678353e-05,
         1.23126507e+00,   6.11834821e-07,   5.81006873e-07], dtype=float32), array([  0.00000000e+00,   1.09110403e+00,   5.30077500e-07,
         1.23554718e+00,   5.81069970e-10,   5.01343467e-10], dtype=float32), array([  3.15853940e-30,   1.08839643e+00,   1.12433918e-04,
         1.20230818e+00,   1.00351338e-06,   9.31600880e-07], dtype=float32), array([  2.20162081e